<a href="https://colab.research.google.com/github/paulosenra/projeto_fakenews/blob/main/fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
from google.colab import userdata

# Obtenha sua API Key do Google Generative AI
api_key = userdata.get('secret_key')

# Configurações do Google Generative AI
genai.configure(api_key=api_key)

# Configuração do Modelo
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}
safety_settings = {
    "HATE": "BLOCK_NONE",
    "HARASSMENT": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

def verificar_noticia(noticia):
    # Lista de fontes confiáveis
    fontes_confiaveis = [
        "https://g1.globo.com/",
        "https://www.cnnbrasil.com.br/",
        "https://www.bbc.com/portuguese"
    ]

    # Usar o modelo Generative AI para analisar a notícia
    model = genai.GenerativeModel(
        model_name="gemini-pro",
        generation_config=generation_config,
        safety_settings=safety_settings,
    )
    chat = model.start_chat(history=[])
    response = chat.send_message(f"Analise a veracidade da seguinte notícia: {noticia}")
    analise_ai = response.text

    # Busca por notícias relacionadas em fontes confiáveis
    noticias_encontradas = []
    for fonte in fontes_confiaveis:
        url = fonte + "/busca/?q=" + noticia
        resposta = requests.get(url)
        if resposta.status_code == 200:
            soup = BeautifulSoup(resposta.text, 'html.parser')

            if fonte == "https://g1.globo.com/":
                links = soup.select('.bastian-page .feed-post-link a')
                for link in links:
                    noticias_encontradas.append(link['href'])

            elif fonte == "https://www.cnnbrasil.com.br/":
                links = soup.select('.home--search__list a')
                for link in links:
                    noticias_encontradas.append(link['href'])

            elif fonte == "https://www.bbc.com/portuguese":
                links = soup.select('.hard-news-media__summary a')
                for link in links:
                    noticias_encontradas.append("https://www.bbc.com" + link['href'])

    if noticias_encontradas:
        return f"Análise do Google Gemini: {analise_ai}\n\nEncontrei as seguintes notícias relacionadas em fontes confiáveis:\n{chr(10).join(noticias_encontradas)}"
    else:
        return f"Análise do Google Gemini: {analise_ai}\n\nNão encontrei informações sobre essa notícia em fontes confiáveis. Recomendo que você verifique a fonte da informação e busque por outras fontes confiáveis."

# Exemplo de uso
noticia = input("Digite a notícia que você quer verificar: ")
resultado = verificar_noticia(noticia)
print(resultado)

# Loop para usar o código várias vezes
while True:
    noticia = input("Digite a notícia que você quer verificar (ou 'fim' para sair): ")
    if noticia.lower() == "fim":
        break
    resultado = verificar_noticia(noticia)
    print(resultado)